# Model Training

In [26]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import re
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
import scipy.stats as stats
from keras.preprocessing.sequence import pad_sequences
from itertools import product
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking

## Load the datasets

In [4]:
# List of datasets (based on the filiters in the proposal)
df_list = ['LT009.csv', 'LT010.csv', 'LT011.csv', 
            'LT012.csv', 'LT014.csv', 'LT016.csv', 'LT017.csv', 
            'LT018.csv', 'LT021.csv', 'SMB001.csv', 'SMB002.csv', 
            'SMB005.csv', 'SMB006.csv', 'SMB007.csv',
            'SMB011.csv', 'SMB012.csv']

In [5]:
# Load all datasets into a dictionary
dataframes = {}
for dataset in df_list:
    var_name = dataset.replace(".csv", "")
    dataframes[var_name] = pd.read_csv("../data/" + dataset)

In [6]:
# Combine all datasets into one DataFrame
df = pd.concat(dataframes.values(), ignore_index=True)

In [7]:
# Split the dataset by species
lt_df = df[df['species'] == 'lakeTrout']
smb_df = df[df['species'] == 'smallmouthBass']

In [8]:
# Convert 'dateProcessed' to proper datetime
df["dateProcessed"] = pd.to_datetime(df["dateProcessed"])

# Convert 'Ping_time' to a time format
df["Ping_time"] = pd.to_datetime(df["Ping_time"].str.strip(), format="%H:%M:%S.%f").dt.time

# Merge dateProcessed (date) and Ping_time (time) into one datetime column
df["Ping_time"] = df.apply(lambda row: pd.Timestamp.combine(row["dateProcessed"], row["Ping_time"]), axis=1)

# Verify output
print(df[["dateProcessed", "Ping_time"]])

     dateProcessed               Ping_time
0       2022-07-26 2022-07-26 15:01:17.016
1       2022-07-26 2022-07-26 15:01:17.220
2       2022-07-26 2022-07-26 15:01:19.119
3       2022-07-26 2022-07-26 15:01:19.220
4       2022-07-26 2022-07-26 15:04:37.217
...            ...                     ...
6080    2022-07-28 2022-07-28 00:17:22.964
6081    2022-07-28 2022-07-28 00:17:25.964
6082    2022-07-28 2022-07-28 00:17:26.163
6083    2022-07-28 2022-07-28 00:17:40.564
6084    2022-07-28 2022-07-28 00:17:40.764

[6085 rows x 2 columns]


In [9]:
# Use regex to select columns that start with "F" followed by a number
f_number_cols = [col for col in df.columns if re.match(r'^F\d+(\.\d+)?$', col)]

# Filter the dataset to keep only these columns
df_filtered = df[["fishNum", "species", "dateProcessed", "Ping_time"] + f_number_cols]

In [10]:
# Extract unique fish IDs
fish_ids = df_filtered['fishNum'].unique()

# Store results
accuracies = []


In [11]:
fish_ids

array(['LT009', 'LT010', 'LT011', 'LT012', 'LT014', 'LT016', 'LT017',
       'LT018', 'LT021', 'SMB001', 'SMB002', 'SMB005', 'SMB006', 'SMB007',
       'SMB011', 'SMB012'], dtype=object)

In [12]:
# Compute variance of frequency features
variance = df_filtered[f_number_cols].var()

In [13]:
# Identify low-variance features (threshold = 0.01 for now)
low_variance_features = variance[variance < 0.01].index

# Compute correlation matrix
correlation_matrix = df_filtered[f_number_cols].corr().abs()

In [14]:
correlation_matrix

,F45,F45.5,F46,F46.5,F47,F47.5,F48,F48.5,F49,F49.5,...,F255.5,F256,F256.5,F257,F257.5,F258,F258.5,F259,F259.5,F260
F45,1.000000,0.925578,0.817663,0.698382,0.616939,0.568117,0.525920,0.486998,0.455559,0.421105,...,0.254512,0.267778,0.277292,0.285232,0.279727,0.279601,0.292153,0.287643,0.286957,0.287015
F45.5,0.925578,1.000000,0.938818,0.820021,0.715581,0.639404,0.580177,0.533465,0.498937,0.462027,...,0.252607,0.264700,0.273301,0.285709,0.281111,0.282042,0.294522,0.291122,0.283186,0.285507
F46,0.817663,0.938818,1.000000,0.937683,0.835332,0.738792,0.656418,0.593296,0.548231,0.506709,...,0.255235,0.267294,0.271794,0.287933,0.289571,0.288128,0.301806,0.300312,0.287714,0.288848
F46.5,0.698382,0.820021,0.937683,1.000000,0.937943,0.837913,0.737283,0.654266,0.597423,0.545035,...,0.252160,0.260503,0.264859,0.281798,0.286985,0.286143,0.299438,0.300488,0.286206,0.286031
F47,0.616939,0.715581,0.835332,0.937943,1.000000,0.938575,0.835795,0.735695,0.658888,0.594723,...,0.247361,0.251672,0.256961,0.277361,0.284927,0.287256,0.298993,0.302613,0.287437,0.284230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F258,0.279601,0.282042,0.288128,0.286143,0.287256,0.292597,0.291563,0.279886,0.268457,0.259397,...,0.426520,0.452107,0.517056,0.649360,0.834036,1.000000,0.834931,0.651949,0.500447,0.449042
F258.5,0.292153,0.294522,0.301806,0.299438,0.298993,0.305553,0.305209,0.291913,0.281806,0.272266,...,0.468390,0.453696,0.461742,0.523377,0.637194,0.834931,1.000000,0.851780,0.644244,0.529446
F259,0.287643,0.291122,0.300312,0.300488,0.302613,0.305486,0.300174,0.286790,0.276507,0.265033,...,0.509008,0.480708,0.462172,0.476601,0.520066,0.651949,0.851780,1.000000,0.841404,0.664431
F259.5,0.286957,0.283186,0.287714,0.286206,0.287437,0.287280,0.278982,0.267312,0.261178,0.251218,...,0.512254,0.504005,0.474280,0.447740,0.440457,0.500447,0.644244,0.841404,1.000000,0.837790


In [15]:
# Identify highly correlated features (threshold = 0.95)
high_correlation_pairs = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        if correlation_matrix.iloc[i, j] > 0.95:
            high_correlation_pairs.add((correlation_matrix.columns[i], correlation_matrix.columns[j]))

# Select one feature per correlated group
selected_features = set(f_number_cols) - set(low_variance_features)
for f1, f2 in high_correlation_pairs:
    if f2 in selected_features:  # Remove the second feature of the pair
        selected_features.remove(f2)

# Display the number of features before and after reduction
len_before = len(f_number_cols)
len_after = len(selected_features)

len_before, len_after, list(selected_features)[:10]  # Show first 10 selected features as a preview


(426,
 394,
 ['F208',
  'F170',
  'F256.5',
  'F140.5',
  'F194',
  'F200',
  'F204',
  'F144.5',
  'F106',
  'F149'])

In [16]:
# Encode species as numerical labels
df_filtered['species_label'] = df_filtered['species'].astype('category').cat.codes

# Further feature selection using Random Forest
X = df_filtered[list(selected_features)]
y = df_filtered['species_label']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train a Random Forest classifier to get feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_scaled, y)

# Get top 30 most important features
feature_importances = pd.Series(rf.feature_importances_, index=selected_features)
top_features_rf = feature_importances.nlargest(30).index
X_rf_selected = df_filtered[top_features_rf]

# Further reduction using PCA
pca = PCA(n_components=10)  # Reduce to 10 principal components
X_pca = pca.fit_transform(X_scaled)

# Show selected features and variance explained by PCA
top_features_rf, pca.explained_variance_ratio_

/var/folders/46/_qfvxk655mzdxw5h9v4z7ybw0000gn/T/ipykernel_27685/630912038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['species_label'] = df_filtered['species'].astype('category').cat.codes


(Index(['F49', 'F59.5', 'F76', 'F48.5', 'F65.5', 'F47', 'F80', 'F60', 'F47.5',
        'F66', 'F60.5', 'F48', 'F59', 'F80.5', 'F49.5', 'F76.5', 'F81', 'F79.5',
        'F46.5', 'F46', 'F78', 'F79', 'F70', 'F45.5', 'F56.5', 'F81.5', 'F69',
        'F77', 'F69.5', 'F50.5'],
       dtype='object'),
 array([0.30778148, 0.04256834, 0.02358369, 0.02129603, 0.01929679,
        0.01718872, 0.01644715, 0.0151755 , 0.01453064, 0.01416199]))

## Leave-One-Fish-Out (LOFO) Cross-Validation
### Model Training

### Logistic Regreesion

In [17]:
# LOFO Setup
groups = df_filtered['fishNum']
logo = LeaveOneGroupOut()

# Models
models = {
    "RandomForest_Selected": LogisticRegression(class_weight="balanced", max_iter=500, random_state=42),
    "PCA_Transformed": LogisticRegression(class_weight="balanced", max_iter=500, random_state=42)
}

# Store results
results = {model: {"accuracy": [], "roc_auc": []} for model in models}

# LOFO Training and Evaluation
for train_idx, test_idx in logo.split(df_filtered, df_filtered['species_label'], groups):
    # Split data
    X_train_rf, X_test_rf = X_rf_selected.iloc[train_idx], X_rf_selected.iloc[test_idx]
    X_train_pca, X_test_pca = X_pca[train_idx], X_pca[test_idx]
    y_train, y_test = df_filtered.loc[train_idx, 'species_label'], df_filtered.loc[test_idx, 'species_label']

    # Standardize RF features
    scaler_rf = StandardScaler()
    X_train_rf_scaled = scaler_rf.fit_transform(X_train_rf)
    X_test_rf_scaled = scaler_rf.transform(X_test_rf)

    # Train & evaluate models
    for model_name, model in models.items():
        if model_name == "RandomForest_Selected":
            X_train, X_test = X_train_rf_scaled, X_test_rf_scaled
        else:
            X_train, X_test = X_train_pca, X_test_pca

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Store accuracy
        results[model_name]["accuracy"].append(accuracy_score(y_test, y_pred))


# Aggregate results
summary_results = {
    model: {
        "Mean Accuracy": np.mean(results[model]["accuracy"]),
    } for model in results
}

# Convert to DataFrame
summary_df = pd.DataFrame(summary_results).T

# Display Results
print(summary_df)

                       Mean Accuracy
RandomForest_Selected       0.657316
PCA_Transformed             0.621657


### XGB Model

In [18]:
# Initialize XGBoost model
xgb_model = xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss", random_state=42)

# Store results for XGBoost
results_xgb = {"accuracy": []}

# LOFO Training and Evaluation for XGBoost
for train_idx, test_idx in logo.split(df_filtered, df_filtered['species_label'], groups):
    # Split data
    X_train, X_test = X_rf_selected.iloc[train_idx], X_rf_selected.iloc[test_idx]
    y_train, y_test = df_filtered.loc[train_idx, 'species_label'], df_filtered.loc[test_idx, 'species_label']

    # Standardize features
    scaler_rf = StandardScaler()
    X_train_scaled = scaler_rf.fit_transform(X_train)
    X_test_scaled = scaler_rf.transform(X_test)

    # Train XGBoost model
    xgb_model.fit(X_train_scaled, y_train)

    # Predict and evaluate
    y_pred = xgb_model.predict(X_test_scaled)

    # Store accuracy
    results_xgb["accuracy"].append(accuracy_score(y_test, y_pred))

# Aggregate results
summary_xgb = {
    "Mean Accuracy": np.mean(results_xgb["accuracy"]),
}

# Convert to DataFrame and display results
summary_df_xgb = pd.DataFrame([summary_xgb])
print(summary_df_xgb)


   Mean Accuracy
0       0.554333


### Hyperparameter tuning for XGBoost

In [19]:
# Define reduced hyperparameter grid for XGBoost
param_dist = {
    "n_estimators": [100, 300, 500],  # Number of trees
    "max_depth": [3, 6, 10],  # Depth of each tree
    "learning_rate": [0.01, 0.1, 0.3],  # Step size shrinkage
    "subsample": [0.6, 0.8, 1.0],  # Fraction of data used per tree
    "colsample_bytree": [0.6, 0.8, 1.0],  # Fraction of features used per tree
}

# Initialize XGBoost model (without deprecated parameter)
xgb_model = xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss", random_state=42)

# Perform Randomized Search with 5-fold cross-validation
random_search = RandomizedSearchCV(
    xgb_model, param_distributions=param_dist, n_iter=20, cv=5, scoring="accuracy", n_jobs=-1, verbose=1, random_state=42
)
random_search.fit(X_rf_selected, df_filtered['species_label'])

# Get best parameters and best accuracy
best_params = random_search.best_params_
best_accuracy = random_search.best_score_

# Display results
summary_xgb_tuning = {
    "Best Accuracy": best_accuracy,
    "Best Parameters": best_params
}

summary_df_xgb_tuning = pd.DataFrame([summary_xgb_tuning])
print(summary_df_xgb_tuning)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
   Best Accuracy                                    Best Parameters
0       0.716187  {'subsample': 0.6, 'n_estimators': 500, 'max_d...


### Use SMOTE to balance data

In [20]:
# Extract higher-level features for each fish
grouped = df_filtered.groupby("fishNum")

# Define functions for feature extraction
def extract_features(group):
    features = {}
    for col in top_features_rf:
        values = group[col].values
        features[f"{col}_mean"] = np.mean(values)
        features[f"{col}_std"] = np.std(values)
        features[f"{col}_skew"] = stats.skew(values)
        features[f"{col}_kurtosis"] = stats.kurtosis(values)
    return pd.Series(features)

# Apply feature extraction
df_features = grouped.apply(extract_features).reset_index()

# Merge species label back
df_features = df_features.merge(df_filtered[['fishNum', 'species_label']].drop_duplicates(), on="fishNum")

# Prepare feature matrix and labels
X_new = df_features.drop(columns=["fishNum", "species_label"])
y_new = df_features["species_label"]
groups = df_features["fishNum"]

# Standardize new features
scaler_new = StandardScaler()
X_new_scaled = scaler_new.fit_transform(X_new)

# Initialize LOFO cross-validation
logo = LeaveOneGroupOut()

# Store results for XGBoost with SMOTE
results_xgb_smote = {"accuracy": []}

# LOFO Training and Evaluation with SMOTE
for train_idx, test_idx in logo.split(X_new_scaled, y_new, groups):
    # Split data
    X_train, X_test = X_new_scaled[train_idx], X_new_scaled[test_idx]
    y_train, y_test = y_new.iloc[train_idx], y_new.iloc[test_idx]

    # Apply SMOTE to balance the training set
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Train XGBoost model
    xgb_model_smote = xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss", random_state=42)
    xgb_model_smote.fit(X_train_resampled, y_train_resampled)

    # Predict and evaluate
    y_pred = xgb_model_smote.predict(X_test)
    results_xgb_smote["accuracy"].append(accuracy_score(y_test, y_pred))

# Aggregate results
summary_smote_lofo = {
    "Mean Accuracy After SMOTE (LOFO)": np.mean(results_xgb_smote["accuracy"])
}

# Convert to DataFrame and display results
summary_df_smote_lofo = pd.DataFrame([summary_smote_lofo])
print(summary_df_smote_lofo)

/var/folders/46/_qfvxk655mzdxw5h9v4z7ybw0000gn/T/ipykernel_27685/1967233190.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_features = grouped.apply(extract_features).reset_index()


   Mean Accuracy After SMOTE (LOFO)
0                            0.5625


In [21]:
# Store results for Random Forest with SMOTE
results_rf_smote = {"accuracy": []}

# LOFO Training and Evaluation with SMOTE for Random Forest
for train_idx, test_idx in logo.split(X_new_scaled, y_new, groups):
    # Split data
    X_train, X_test = X_new_scaled[train_idx], X_new_scaled[test_idx]
    y_train, y_test = y_new.iloc[train_idx], y_new.iloc[test_idx]

    # Apply SMOTE to balance the training set
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Train Random Forest model
    rf_model_smote = RandomForestClassifier(n_estimators=500, random_state=42)
    rf_model_smote.fit(X_train_resampled, y_train_resampled)

    # Predict and evaluate
    y_pred = rf_model_smote.predict(X_test)
    results_rf_smote["accuracy"].append(accuracy_score(y_test, y_pred))

# Aggregate results
summary_smote_rf_lofo = {
    "Mean Accuracy After SMOTE (LOFO) - RF": np.mean(results_rf_smote["accuracy"])
}

# Convert to DataFrame and display results
summary_df_smote_rf_lofo = pd.DataFrame([summary_smote_rf_lofo])
print(summary_df_smote_rf_lofo)

   Mean Accuracy After SMOTE (LOFO) - RF
0                                 0.6875


## ML models performed not well, they did not incorporate time series nature in this dataset. We need to consider using models designed for time-series.
Besides, leave one fish out makes the test set to have only one species. It's better to leave a pair of fish out instead.

### LSTM

In [22]:
# Reshape the dataset for LSTM

# Sort by fish and time to maintain sequence order
df_lstm = df_filtered.sort_values(by=["fishNum", "Ping_time"])

# Select frequency feature columns
frequency_cols = df_lstm.columns[4:-1]

# Normalize the frequency features
scaler = StandardScaler()
df_lstm[frequency_cols] = scaler.fit_transform(df_lstm[frequency_cols])

# Group by fish to create sequences
grouped = df_lstm.groupby("fishNum")

# Convert each fish's time series into a numpy array
fish_sequences = []
fish_labels = []
fish_nums = []

for fish_id, group in grouped:
    fish_sequences.append(group[frequency_cols].values)  # Time-series features
    fish_labels.append(group["species"].iloc[0])   # Label for the fish
    fish_nums.append(fish_id)  # Track which fish

# Convert to numpy arrays (keeping sequences as objects for now)
fish_sequences = np.array(fish_sequences, dtype=object)  # Different lengths
fish_labels = np.array(fish_labels)

# Find max sequence length for padding
max_timesteps = max([seq.shape[0] for seq in fish_sequences])

# Display key info
max_timesteps, len(fish_sequences), fish_sequences[0].shape


(1379, 16, (320, 426))

In [23]:
# Pad sequences to make all fish have the same length
fish_sequences_padded = pad_sequences(fish_sequences, maxlen=max_timesteps, dtype="float32", padding="post", truncating="post")

# Convert species labels to numerical encoding (binary classification)
species_mapping = {species: idx for idx, species in enumerate(np.unique(fish_labels))}
fish_labels_encoded = np.array([species_mapping[label] for label in fish_labels])

# Display the shape of the final LSTM input data
fish_sequences_padded.shape, fish_labels_encoded.shape

((16, 1379, 426), (16,))

In [25]:
# Create lists of fish numbers for each species
lt_fish = [fish for fish, label in zip(fish_nums, fish_labels_encoded) if label == 0]  # LT species
smb_fish = [fish for fish, label in zip(fish_nums, fish_labels_encoded) if label == 1]  # SMB species

# Generate all possible (LT, SMB) pairs for LOPO
lopo_pairs = list(product(lt_fish, smb_fish))

# Display the number of LOPO folds created and the first few pairs
len(lopo_pairs), lopo_pairs[:5]

(63,
 [('LT009', 'SMB001'),
  ('LT009', 'SMB002'),
  ('LT009', 'SMB005'),
  ('LT009', 'SMB006'),
  ('LT009', 'SMB007')])

In [27]:
# Define LSTM model structure
def build_lstm_model(input_shape):
    model = Sequential([
        Masking(mask_value=0.0, input_shape=input_shape),  # Mask padding values
        LSTM(64, return_sequences=True),  # LSTM layer with 64 units
        LSTM(32),  # Another LSTM layer
        Dense(16, activation="relu"),  # Fully connected layer
        Dense(1, activation="sigmoid")  # Output layer (binary classification)
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Define input shape for the model (timesteps, features)
input_shape = (max_timesteps, fish_sequences_padded.shape[2])

# Build the LSTM model
lstm_model = build_lstm_model(input_shape)

# Display model summary
lstm_model.summary()


/Users/scarlett/opt/miniconda3/envs/sta2453/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 1379, 426)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 1379, 64)       │       125,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,657 (541.63 KB)

 Trainable params: 138,657 (541.63 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Store results for LSTM with LOPO
results_lstm_lopo = {"accuracy": []}

# LOPO Training and Evaluation for LSTM
for lt_fish, smb_fish in lopo_pairs:
    # Identify training and test indices
    test_indices = [fish_nums.index(lt_fish), fish_nums.index(smb_fish)]
    train_indices = [i for i in range(len(fish_nums)) if i not in test_indices]

    # Split data
    X_train, X_test = fish_sequences_padded[train_indices], fish_sequences_padded[test_indices]
    y_train, y_test = fish_labels_encoded[train_indices], fish_labels_encoded[test_indices]

    # Train LSTM model
    lstm_model = build_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=2, verbose=0)

    # Predict and evaluate
    y_pred = (lstm_model.predict(X_test) > 0.5).astype("int32").flatten()
    results_lstm_lopo["accuracy"].append(accuracy_score(y_test, y_pred))

# Aggregate results
summary_lstm_lopo = {
    "Mean Accuracy After LOPO (LSTM)": np.mean(results_lstm_lopo["accuracy"])
}

# Convert to DataFrame and display results
summary_df_lstm_lopo = pd.DataFrame([summary_lstm_lopo])
print(summary_df_lstm_lopo) 

/Users/scarlett/opt/miniconda3/envs/sta2453/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 610ms/step


/Users/scarlett/opt/miniconda3/envs/sta2453/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step


/Users/scarlett/opt/miniconda3/envs/sta2453/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 826ms/step


/Users/scarlett/opt/miniconda3/envs/sta2453/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 653ms/step


/Users/scarlett/opt/miniconda3/envs/sta2453/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 